## Imports

In [1]:
import json
import glob
import subprocess
from collections import OrderedDict
import itertools
import numpy as np
import matplotlib.pyplot as plt

import librosa
from IPython.display import Audio as ipy_audio
from IPython.core.display import display

from quicktranscribe import tonic, pitch, wave, kde
# from mogra import tonnetz
from mogra.datatypes import Swar, normalize_frequency, ratio_to_swar, SWAR_BOUNDARIES

In [3]:
# syntonic comma in the 0 to 1 scale
SYNTONIC_COMMA = (librosa.hz_to_midi(220*81/80) - librosa.hz_to_midi(220))/12

## Util Functions

In [6]:
def fetch_audio(ra, savedir):
    for raag, vv in ra.items():
        for artist, url in vv.items():
            command = f"/opt/homebrew/bin/yt-dlp {url} -f 'ba' -x --audio-format 'mp3' --ffmpeg-location /opt/homebrew/bin/ffmpeg -P {savedir}/ -o {raag}-{artist}.mp3"
            result = subprocess.run(command, shell=True, capture_output=True)
            print(result.stdout.decode())
            if len(result.stderr) > 0:
                print("Error:", result.stderr.decode())

In [7]:
def annotate_tonic(track_path, plot=False):
    DEFAULT_TONIC = 220
    np.set_printoptions(suppress=True)
    
    start=7*60
    end=8*60
    y_stereo, sr = wave.read_audio_section(track_path + ".mp3", start, end)
    y_sample = librosa.to_mono(y_stereo.T)
    
    kde_sample = kde.extract(y_sample, sr=sr, tonic=DEFAULT_TONIC)
    peaks, _ = kde.prominence_based_peak_finder(kde_sample, prominence=0.005)
    print(peaks)

    if plot:
        plt.plot(np.linspace(0, 12, len(kde_sample)), kde_sample, color="teal")
        plt.plot(np.array(peaks) * 12/len(kde_sample), kde_sample[peaks], "o", markersize="3", color="orange")
    
    display(ipy_audio(y_sample, rate=sr))
    input("hear the audio and press any key to continue")
    
    peaks = sorted(peaks, key=lambda x: kde_sample[x], reverse=True)
    found_tonic = False
    for peak in peaks:
        # generate a sine wave of the peak frequency and play it
        fpeak = librosa.midi_to_hz(librosa.hz_to_midi(DEFAULT_TONIC) + 12 * peak / len(kde_sample))
        ypeak = librosa.tone(fpeak, duration=3)
        display(ipy_audio(ypeak, rate=sr))
        ft = input("Is this the tonic? (y/n): ")
        if ft == "y":
            found_tonic = True
            break
    
    if not found_tonic:
        print("No tonic found")
        return None
    
    # write tonic to file
    tonic.write_tonic(track_path + ".ctonic.txt", fpeak)

In [8]:
def read_sample_and_tonic(track_path):
    
    ctonic = tonic.read_tonic(track_path + ".ctonic.txt")
    # metadata = tonic.read_metadata(track_path + ".json")
    # pitch_annotations, aps = pitch.read_pitch(track_path + ".pitch.txt")
    
    # # full audio
    # y_sample, sr = wave.get_audio(track_path + ".mp3")

    # # 5-minute sample
    start=5*60
    end=10*60
    y_stereo, sr = wave.read_audio_section(track_path + ".mp3", start, end)
    y_sample = librosa.to_mono(y_stereo.T)
    # ipy_audio(data=y_sample, rate=sr)
    
    return y_sample, sr, ctonic

## Raags & Tracks

In [9]:
DATA_DIR = "concrete-demo"

In [ ]:
raag_theoretical = {
    "Bhoop": [1, 10/9, 5/4, 3/2, 5/3],
    "Yaman": [1, 9/8, 5/4, 45/32, 3/2, 27/16, 15/8],
}
for raag, vv in raag_theoretical.items():
    raag_theoretical[raag] = [normalize_frequency(f) for f in vv]
    print(raag)
    for ii in range(len(raag_theoretical[raag])):
        print(f"{ratio_to_swar(raag_theoretical[raag][ii])}: {np.round(raag_theoretical[raag][ii], 6)})")

In [11]:
raags_and_artists = {
    "Bhoop": {
        "MilindRaikar": "https://www.youtube.com/watch?v=9a7NhReDWy8",
        "DKDatar": "https://www.youtube.com/watch?v=z5RemO4d41o",
    },
    "Yaman" : {
        "KalaRamnath": "https://www.youtube.com/watch?v=kvTqtXP6lmo",
        "NandiniShankar": "https://www.youtube.com/watch?v=ldS89LPpQ_w",
        "DKDatar": "https://www.youtube.com/watch?v=EemtViN7zM8",
    }
}

## tonnetz

In [5]:
from dataclasses import dataclass
from collections import OrderedDict
from enum import Enum
from typing import List, Dict, Tuple
import itertools

import plotly.graph_objects as go
import numpy as np
from mogra.datatypes import normalize_frequency, ratio_to_swar, Swar

OCCUR_FREQ_THRESHOLD = 0.04  # a normalized probability below this => ignore this note

In [6]:
class EFGenus:
    def __init__(self, primes=[3, 5, 7], powers=[0, 0, 0]) -> None:
        self.primes = primes
        self.powers = powers
    
    @classmethod
    def from_list(cls, genus_list: List):
        primes = []
        powers = []
        for new_prime in genus_list:
            if len(primes) > 0:
                assert new_prime >= primes[-1]
                if new_prime == primes[-1]:
                    powers[-1] += 1
                else:
                    primes.append(new_prime)
                    powers.append(1)
            else:
                primes.append(new_prime)
                powers.append(1)
                
        return cls(primes, powers)

In [63]:
class Tonnetz:
    def __init__(self, genus) -> None:
        if len(genus.primes) > 3:
            print("cannot handle more than 3 dimensions")
            return

        self.primes = genus.primes
        self.powers = genus.powers
        
        ranges = []
        for prime, power in zip(genus.primes, genus.powers):
            ranges.append(range(-power, power+1))
        self.node_coordinates = list(itertools.product(*ranges))
        
        self.assign_coords3d()
        self.assign_notes()
    
    def prep_plot(self, figure):
        camera = dict(
            up=dict(x=0, y=0, z=1),
            center=dict(x=0, y=0, z=0),
            eye=dict(x=1.25, y=-1.25, z=1.25)
        )
        figure.update_layout(scene_aspectmode="data", scene_camera=camera)
        figure.update_layout(
            scene=dict(
                xaxis_title = self.primes[0] if len(self.primes) > 0 else "null",
                yaxis_title = self.primes[1] if len(self.primes) > 1 else "null",
                zaxis_title = self.primes[2] if len(self.primes) > 2 else "null",
            ),
        )
        return figure
    
    def frequency_from_coord(self, coords):
        ff = 1
        for ii, cc in enumerate(coords):
            ff *= self.primes[ii]**cc
        return ff
    
    def assign_coords3d(self):
        coords = list(zip(*self.node_coordinates))
        # Coordinates for Plotly Scatter3d
        self.coords3d = {i: [0] * len(self.node_coordinates) for i in range(3)}
        for i, coords in enumerate(coords):
            if i < len(coords):
                self.coords3d[i] = coords
    
    def assign_notes(self):
        self.node_frequencies = [
            normalize_frequency(self.frequency_from_coord(nc))
            for nc in self.node_coordinates
        ]
        self.node_names = [
            ratio_to_swar(nf)
            for nf in self.node_frequencies
        ]
    
    def plot(self):        
        # Create the 3D scatter plot
        fig = go.Figure(data=[go.Scatter3d(
            x=self.coords3d[0],
            y=self.coords3d[1],
            z=self.coords3d[2],
            mode="text+markers",
            marker=dict(size=12, symbol="circle"),
            marker_color=["midnightblue" for mm in self.node_names],
            text=self.node_names,
            textposition="middle center",
            textfont=dict(family="Overpass", size=10, color="white"),
        )])
        
        fig = self.prep_plot(fig)
        fig.show()

    def plot_swar_set(self, swar_set):
        fig = go.Figure(data=[go.Scatter3d(
            x=self.coords3d[0],
            y=self.coords3d[1],
            z=self.coords3d[2],
            mode="text+markers",
            marker=dict(
                size=12,
                symbol="circle",
                color=["gold" if mm in swar_set else "midnightblue" for mm in self.node_names]
            ),
            text=self.node_names,
            textposition="middle center",
            textfont=dict(family="Overpass", size=10, color="white"),
        )])
        
        fig = self.prep_plot(fig)
        fig.show()
    
    def plot_swar_hist(self, swar_set, swar_occur):
        fig = go.Figure(data=[go.Scatter3d(
            x=self.coords3d[0],
            y=self.coords3d[1],
            z=self.coords3d[2],
            mode="text+markers",
            marker=dict(
                size=[5 if mm not in swar_set else 100 * swar_occur[swar_set.index(mm)] for mm in self.node_names],
                symbol="circle",
                color=["gold" if mm in swar_set else "midnightblue" for mm in self.node_names]
            ),
            text=self.node_names,
            textposition="middle center",
            textfont=dict(
                # family="Overpass",
                size=[10 if mm not in swar_set else 30 * swar_occur[swar_set.index(mm)] for mm in self.node_names],
                color="dimgray"
            ),
        )])
        
        fig = self.prep_plot(fig)
        fig.show()

    def plot_cone(self):
        """
        tonnetz + folded frequency heights
        """
        assert len(self.primes) == 2
        # seq = np.argsort(self.node_frequencies)
        # breakpoint()
        fig = go.Figure(data=[go.Scatter3d(
            x=self.coords3d[0],
            y=self.coords3d[1],
            z=self.node_frequencies,
            mode="text+markers",
            marker=dict(size=12, symbol="circle"),
            marker_color=["midnightblue" for mm in self.node_names],
            text=self.node_names,
            textposition="middle center",
            textfont=dict(family="Overpass", size=10, color="white"),
        )])
        fig = self.prep_plot(fig)
        # fig.update_zaxes(title_text="frequency ratio", type="log")
        fig.update_layout(
            scene=dict(
                xaxis_title = self.primes[0] if len(self.primes) > 0 else "null",
                yaxis_title = self.primes[1] if len(self.primes) > 1 else "null",
                zaxis_title = self.primes[2] if len(self.primes) > 2 else "frequency",
                zaxis_type = "log"
            ),
        )
        fig.show()
        
    def plot1d(self):
        """
        post octave-folding
        """
        seq = np.argsort(self.node_frequencies)
        fig = go.Figure(data=go.Scatter(
            x=[
                sum([np.log(self.primes[ii])*pows[ii] for ii in range(len(self.primes))])
                for pows in np.array(self.node_coordinates)[seq]
            ],  # hints at the power complexity
            y=np.array(self.node_frequencies)[seq],  # just the sorted frequencies
            mode="markers+text",
            marker=dict(size=14, symbol="circle"),
            marker_color=["midnightblue" for mm in np.array(self.node_names)[seq]],
            text=np.array(self.node_names)[seq],
            textposition="middle center",
            textfont=dict(family="Overpass", size=12, color="white"),
        ))
        fig.update_yaxes(title_text="frequency ratio", type="log")
        fig.update_layout(autosize=False, width=700, height=700)
        fig.layout.yaxis.scaleanchor="x"
        fig.show()
    
    def get_swar_options(self, swar):
        swar_node_indices = [nn == swar for nn in self.node_names]
        swar_node_coordinates = np.array(self.node_coordinates)[swar_node_indices]
        return [tuple(nc) for nc in swar_node_coordinates.tolist()], self.primes
    
    def get_neighbors(self, node: List):
        neighbors = []
        for nc in self.node_coordinates:
            if sum(abs(np.array(nc)-np.array(node))) == 1:
                neighbors.append(nc)
        return neighbors

In [69]:
class TonnetzAlgo1:
    def __init__(self, net: Tonnetz) -> None:
        self.net = net
        # hyperparameters
        # TODO(neeraja): replace placeholder penalties
        self.prime_penalties = [np.exp(pp)/np.exp(5) for ii, pp in enumerate(self.net.primes)]
    
    def compute_prime_complexity(self, node):
        # TODO(neeraja): replace placeholder formula
        return sum([abs(node[ii])*self.prime_penalties[ii] for ii in range(len(node))])
        
    def set_pc12(self, pc12_distribution):
        """ assign initial weights to all the nodes
        """
        assert len(pc12_distribution) == 12
        pc12_distribution = pc12_distribution/np.sum(pc12_distribution)
        self.pc12_distribution = pc12_distribution
        self.node_distribution = [
            pc12_distribution[Swar[nn].value]
            for nn in self.net.node_names
        ]
    
    def plot_swar_hist(self):
        fig = go.Figure(data=[go.Scatter3d(
            x=self.net.coords3d[0],
            y=self.net.coords3d[1],
            z=self.net.coords3d[2],
            mode="text+markers",
            text=self.net.node_names,
            textposition="middle center",
            textfont=dict(
                # family="Overpass",
                size=[30 * mm if mm > OCCUR_FREQ_THRESHOLD else 10 for mm in self.node_distribution],
                color="dimgray"
            ),
        )])
        
        fig = self.net.prep_plot(fig)
        fig.show()

    def consolidate_sa(self):
        sa_options, primes = self.net.get_swar_options("S")
        for sa_option in sa_options:
            if (sa_option == np.zeros(len(primes))).all():
                continue
            self.node_distribution[self.net.node_coordinates.index(sa_option)] = 0
    
    def zero_out_below_threshold(self):
        for ii, nn in enumerate(self.net.node_names):
            if self.node_distribution[ii] < OCCUR_FREQ_THRESHOLD:
                self.node_distribution[ii] = 0

    def consolidate_swar(self, swar):
        # get options
        swar_options, primes = self.net.get_swar_options(swar)
        # keep track of scores
        swar_option_scores = {}
        for swar_option in swar_options:
            # get all the neighbors
            nbd = self.net.get_neighbors(swar_option)
            nbd_score = np.sum([self.node_distribution[self.net.node_coordinates.index(nbd_node)] for nbd_node in nbd])
            # compute prime complexity
            prime_complexity = self.compute_prime_complexity(swar_option)
            # TODO(neeraja): replace placeholder formula
            total_score = nbd_score + 1/prime_complexity
            swar_option_scores[swar_option] = total_score
        print(f"options for {swar}: {swar_option_scores}")
        winning_option = max(swar_option_scores, key=swar_option_scores.get)
        print(f"winner for swar {swar}: {winning_option}")
        # zero out the rest
        for swar_option in swar_options:
            if swar_option == winning_option:
                continue
            self.node_distribution[self.net.node_coordinates.index(swar_option)] = 0
        
    def execute(self, plot=True):
        if plot:
            print("initial plot")
            self.plot_swar_hist()

        self.consolidate_sa()
        def sort_nonsa_swars(pc12_distribution):
            thresholded_set = np.where(pc12_distribution > OCCUR_FREQ_THRESHOLD)[0]
            nonsa_set = "".join([Swar(ii).name for ii in thresholded_set if ii != 0])
            nonsa_occur = [pc12_distribution[Swar[swar].value] for swar in nonsa_set]
            decreasing = np.argsort(nonsa_occur)[::-1]
            sorted_nonsa_set = [nonsa_set[i] for i in decreasing]
            return sorted_nonsa_set

        self.zero_out_below_threshold()
        for ss in sort_nonsa_swars(self.pc12_distribution):
            self.consolidate_swar(ss)
            self.plot_swar_hist()

        if plot:
            print("final plot")
            self.plot_swar_hist()

        result = {}
        for nd in self.net.node_coordinates:
            if self.node_distribution[self.net.node_coordinates.index(nd)] > 0:
                result[ratio_to_swar(normalize_frequency(self.net.frequency_from_coord(nd)))] = normalize_frequency(self.net.frequency_from_coord(nd))
        result = OrderedDict(sorted(result.items(), key=lambda x: x[1]))
        return result

In [70]:
class TonnetzAlgo2:
    def __init__(self, net: Tonnetz) -> None:
        self.net = net
        # hyperparameters
        # TODO(neeraja): replace placeholder penalties
        self.prime_penalties = [np.exp(pp)/np.exp(5) for ii, pp in enumerate(self.net.primes)]
    
    def compute_prime_complexity(self, node):
        # TODO(neeraja): replace placeholder formula
        return sum([abs(node[ii])*self.prime_penalties[ii] for ii in range(len(node))])
        
    def set_pc12(self, pc12_distribution):
        """ assign initial weights to all the nodes
        """
        assert len(pc12_distribution) == 12
        pc12_distribution = pc12_distribution/np.sum(pc12_distribution)
        self.pc12_distribution = pc12_distribution
        self.node_distribution = [
            pc12_distribution[Swar[nn].value]
            for nn in self.net.node_names
        ]
    
    def plot_swar_hist(self):
        fig = go.Figure(data=[go.Scatter3d(
            x=self.net.coords3d[0],
            y=self.net.coords3d[1],
            z=self.net.coords3d[2],
            mode="text+markers",
            marker=dict(
                size=100 * np.array(self.node_distribution),
                symbol="circle",
                color=["gold" if mm > OCCUR_FREQ_THRESHOLD else "midnightblue" for mm in self.node_distribution]
            ),
            text=self.net.node_names,
            textposition="middle center",
            textfont=dict(
                # family="Overpass",
                size=[30 * mm if mm > OCCUR_FREQ_THRESHOLD else 10 for mm in self.node_distribution],
                color="dimgray"
            ),
        )])
        
        fig = self.net.prep_plot(fig)
        fig.show()

    def consolidate_sa(self):
        sa_options, primes = self.net.get_swar_options("S")
        for sa_option in sa_options:
            if (sa_option == np.zeros(len(primes))).all():
                continue
            self.node_distribution[self.net.node_coordinates.index(sa_option)] = 0
    
    def zero_out_below_threshold(self):
        for ii, nn in enumerate(self.net.node_names):
            if self.node_distribution[ii] < OCCUR_FREQ_THRESHOLD:
                self.node_distribution[ii] = 0

    def execute(self, plot=True):
        if plot:
            print("initial plot")
            self.plot_swar_hist()

        self.consolidate_sa()
        
        def nonsa_swars(pc12_distribution):
            thresholded_set = np.where(pc12_distribution > OCCUR_FREQ_THRESHOLD)[0]
            nonsa_set = "".join([Swar(ii).name for ii in thresholded_set if ii != 0])
            return nonsa_set

        # get all possible swar options
        options = {}
        for ss in nonsa_swars(self.pc12_distribution):
            possible_options, _ = self.net.get_swar_options(ss)
            options[ss] = possible_options
        
        # iterate over all cartesian products
        iteration_results = []
        for combo_option in itertools.product(*options.values()):
            # add sa to the tuple
            combo_option = [tuple(np.zeros(len(self.net.primes), dtype=int))] + list(combo_option)
            # need a measure of how well-connected this combo_option is
            # for each swar in the combo_option, get the neighbors within the combo_option
            # get the sum of the node_distribution values for these neighbors
            # also get the prime complexity of the combo_option
            nbd_score = 0
            prime_complexity = 0
            for swar in combo_option:
                nbd = self.net.get_neighbors(swar)
                nbd_score += np.sum([
                    self.node_distribution[self.net.node_coordinates.index(nbd_node)] if nbd_node in combo_option else 0
                    for nbd_node in nbd
                ])
                prime_complexity += self.compute_prime_complexity(swar)
            iteration_results.append((combo_option, nbd_score, prime_complexity))
            # print(f"{combo_option}: {nbd_score}, {prime_complexity}")
        
        # get the top 5 iteration_results by least prime_complexity
        print("\n")
        ir = sorted(iteration_results, key=lambda x: x[2])
        [print(f"{combo_option}: {nbd_score}, {prime_complexity}") for combo_option, nbd_score, prime_complexity in ir[:5]]
        # get the top 5 iteration_results by most nbd_score
        print("\n")
        ir = sorted(iteration_results, key=lambda x: x[1], reverse=True)
        [print(f"{combo_option}: {nbd_score}, {prime_complexity}") for combo_option, nbd_score, prime_complexity in ir[:5]]
        # get the top 5 with a combined score
        ALPHA = 0.5  # weight of prime complexity
        score = lambda x: x[1] + ALPHA * 1/x[2]
        print("\n")
        ir = sorted(iteration_results, key=score, reverse=True)
        [print(f"{combo_option}: {nbd_score}, {prime_complexity}") for combo_option, nbd_score, prime_complexity in ir[:5]]

        if plot:
            print("final plot")
            self.plot_swar_hist()

        result = {}
        for nd in self.net.node_coordinates:
            if self.node_distribution[self.net.node_coordinates.index(nd)] > 0:
                result[ratio_to_swar(normalize_frequency(self.net.frequency_from_coord(nd)))] = normalize_frequency(self.net.frequency_from_coord(nd))
        result = OrderedDict(sorted(result.items(), key=lambda x: x[1]))
        return result

In [85]:
class TonnetzAlgo3:
    """ note the different prime penalty formulae """
    def __init__(self, net: Tonnetz) -> None:
        self.net = net
        # hyperparameters
        # TODO(neeraja): replace placeholder penalties
        self.prime_penalties = [np.exp(pp/2)/np.exp(2) for ii, pp in enumerate(self.net.primes)]
    
    def compute_prime_complexity(self, node):
        # TODO(neeraja): replace placeholder formula
        return sum([abs(node[ii])**self.prime_penalties[ii] for ii in range(len(node))])
        
    def set_pc12(self, pc12_distribution):
        """ assign initial weights to all the nodes
        """
        assert len(pc12_distribution) == 12
        pc12_distribution = pc12_distribution/np.sum(pc12_distribution)
        self.pc12_distribution = pc12_distribution
        self.node_distribution = [
            pc12_distribution[Swar[nn].value]
            for nn in self.net.node_names
        ]
    
    def plot_swar_hist(self):
        fig = go.Figure(data=[go.Scatter3d(
            x=self.net.coords3d[0],
            y=self.net.coords3d[1],
            z=self.net.coords3d[2],
            mode="text+markers",
            marker=dict(
                size=100 * np.array(self.node_distribution),
                symbol="circle",
                color=["gold" if mm > OCCUR_FREQ_THRESHOLD else "midnightblue" for mm in self.node_distribution]
            ),
            text=self.net.node_names,
            textposition="middle center",
            textfont=dict(
                # family="Overpass",
                size=[30 * mm if mm > OCCUR_FREQ_THRESHOLD else 10 for mm in self.node_distribution],
                color="dimgray"
            ),
        )])
        
        fig = self.net.prep_plot(fig)
        fig.show()

    def consolidate_sa(self):
        sa_options, primes = self.net.get_swar_options("S")
        for sa_option in sa_options:
            if (sa_option == np.zeros(len(primes))).all():
                continue
            self.node_distribution[self.net.node_coordinates.index(sa_option)] = 0
    
    def zero_out_below_threshold(self):
        for ii, nn in enumerate(self.net.node_names):
            if self.node_distribution[ii] < OCCUR_FREQ_THRESHOLD:
                self.node_distribution[ii] = 0

    def execute(self, plot=True):
        if plot:
            print("initial plot")
            self.plot_swar_hist()

        self.consolidate_sa()
        
        def nonsa_swars(pc12_distribution):
            thresholded_set = np.where(pc12_distribution > OCCUR_FREQ_THRESHOLD)[0]
            nonsa_set = "".join([Swar(ii).name for ii in thresholded_set if ii != 0])
            return nonsa_set

        # get all possible swar options
        options = {}
        for ss in nonsa_swars(self.pc12_distribution):
            possible_options, _ = self.net.get_swar_options(ss)
            options[ss] = possible_options
        
        # iterate over all cartesian products
        iteration_results = []
        for combo_option in itertools.product(*options.values()):
            # add sa to the tuple
            combo_option = [tuple(np.zeros(len(self.net.primes), dtype=int))] + list(combo_option)
            # measure a pairwise, weighted prime complexity
            prime_complexity = 0
            for ii, jj in itertools.combinations(combo_option, 2):
                prime_complexity += self.compute_prime_complexity(tuple(np.array(jj)-np.array(ii))) * self.node_distribution[self.net.node_coordinates.index(ii)] * self.node_distribution[self.net.node_coordinates.index(jj)]
            iteration_results.append((combo_option, prime_complexity))
        
        # get the top 5 iteration_results by least prime_complexity
        print("\n")
        ir = sorted(iteration_results, key=lambda x: x[1])
        [print(f"{combo_option}: {prime_complexity}") for combo_option, prime_complexity in ir[:5]]

        if plot:
            print("final plot")
            self.plot_swar_hist()

        result = {}
        for nd in self.net.node_coordinates:
            if self.node_distribution[self.net.node_coordinates.index(nd)] > 0:
                result[ratio_to_swar(normalize_frequency(self.net.frequency_from_coord(nd)))] = normalize_frequency(self.net.frequency_from_coord(nd))
        result = OrderedDict(sorted(result.items(), key=lambda x: x[1]))
        return result

## Algorithmic Frequencies

In [ ]:
# Mock Bhoop Sample
pc12_sample = np.array([10, 0.5, 4, 0.8, 6.5, 1, 0.5, 4, 0.5, 3, 0, 0.1])
pc12_sample = pc12_sample / np.sum(pc12_sample)
pc12_sample

In [91]:
# g2 = EFGenus.from_list([3,3,3,5,7])
g1 = EFGenus.from_list([3,3,3,3,5,5])
tn = Tonnetz(g1)

In [ ]:
algo3 = TonnetzAlgo3(tn)
algo3.set_pc12(pc12_sample)
tonnetz_swar_set = algo3.execute()

In [67]:
# algo1 = TonnetzAlgo1(tn)
# algo1.set_pc12(pc12_sample)
# tonnetz_swar_set = algo1.execute()

In [73]:
# algo2 = TonnetzAlgo2(tn)
# algo2.set_pc12(pc12_sample)
# tonnetz_swar_set = algo2.execute()

For Bhoop, Re=(2,0) will always have a better score than Re=(-2,1) since the former is next to a P v/s a D, and also has better prime complexity. Ways out of this **while only looking at pc12**:
1. Measure pairwise prime complexity
2. Do 1. with node wieghts, and ditch the nbd scoring since we're already capturing nearness to other notes by weight.

In [ ]:
print("Tonnetz-Opt Frequencies")
for swar in tonnetz_swar_set.keys():
    print(f"{swar}: {tonnetz_swar_set[swar]}")